# ML Fundamentals

## Linear Regression

### From Scratch

In [46]:
import numpy as np
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt

In [68]:
dataset = load_boston()
X = dataset.data
y = dataset.target[:, None] # 'None' adds a dimension; same as np.newaxis
W = np.zeros(X.shape[-1])[:, None]

X.shape, W.shape, y.shape

((506, 13), (13, 1), (506, 1))

In [33]:
def mse(y_hat, y):
    return (1/2)*len(y)*(y_hat-y).pow(2) # having 1/2 makes derivative easier to calculate

In [ ]:
def gradient_descent(X, y, W, alpha=0.1, epochs=100):
    """
    X: examples
    y: ground truth outputs
    W: weights, params, coefs
    alpha: learning rate
    epochs: num iterations
    """
    n = len(X)
    losses = []
    for _ in range(epochs):
        W = W - (alpha/n) * X.T @ (X@W-y) # gradient update with learning rate and loss derivative w.r.t W
        losses.append(mse(X@W, y))
    return (losses, W)

### Using Scikit-Learn 

In [34]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston

In [35]:
dataset = load_boston()
x = dataset.data
y = dataset.target[:, None]

In [39]:
model = LinearRegression().fit(x, y)